In [2]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [3]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [4]:
def compute_hinge_loss(y, X, w):
    loss_vector = 1 - y * X.dot(w)
    return np.sum(loss_vector[loss_vector > 0])

In [5]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    return compute_hinge_loss(y, X, w) + (lambda_ / 2) * w.dot(w)

In [6]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    prediction_vector = X.dot(w)
    predicted_labels = np.where(prediction_vector > 0, 1, -1)
    return np.mean(y == predicted_labels)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [7]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    x_n, y_n = X[n], y[n]
    gradient_hinge = - y_n * x_n if y_n * x_n.T.dot(w) < 1 else np.zeros(w.size)
    return num_examples * gradient_hinge + lambda_ * w 

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [12]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=3050992975970.175
iteration=10000, cost=26393834030.43944
iteration=20000, cost=20631916344.599968
iteration=30000, cost=18623155484.360382
iteration=40000, cost=18029509082.360916
iteration=50000, cost=17045843678.845016
iteration=60000, cost=16295771513.202736
iteration=70000, cost=15886515870.770853
iteration=80000, cost=15535453027.727507
iteration=90000, cost=15049854794.61503
training accuracy = 0.7


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [25]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    g = 1 - y_n * x_n.dot(w) 
    if old_alpha_n == 0:
        alpha[n] = 1
    elif old_alpha_n == 1 and g < 0:
        alpha[n] = 0
    else:
        alpha[n] = 
    if alpha[n] != old_alpha_n:
        w += x_n * y_n * alpha[n] / lambda_
    return w, alpha

In [23]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha) - (lambda_ / 2.) * w.dot(w) # w = (1 / lambda) * X.T.dot(y).dot(alpha)

In [24]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:1809908246720.74365, dual:-367796495.75040, gap:1810276043216.49414
iteration=10000, primal:1376587801595.28931, dual:-16422154277.14716, gap:1393009955872.43652
iteration=20000, primal:660113256876.66284, dual:-34164156057.41211, gap:694277412934.07495
iteration=30000, primal:360166666157.77966, dual:-51424938878.42730, gap:411591605036.20697
iteration=40000, primal:780190294974.41370, dual:-55860518209.14948, gap:836050813183.56323
iteration=50000, primal:585772833590.73267, dual:-58229300796.00300, gap:644002134386.73572
iteration=60000, primal:775170376663.30115, dual:-62736003810.55486, gap:837906380473.85596
iteration=70000, primal:688967527857.25781, dual:-62774285247.89076, gap:751741813105.14856
iteration=80000, primal:648252848729.36914, dual:-63414588976.91785, gap:711667437706.28699
iteration=90000, primal:1140215800601.18481, dual:-64087225269.86260, gap:1204303025871.04736
training accuracy = 0.713
